In [26]:
from DMDana.lib.DMDparser import DMD,read_text_from_file
from DMDana.lib.constant import *
from multiprocessing import Pool
from functools import wraps
import pandas as pd
import numpy as np
import os
import logging
import DMDana.lib.DMDparser as DMDparser
import shutil
from DMDana.do.occup_time import fermi 
excel_index_shift=2

def read_database(path):
    return pd.read_excel(path)
def save_database(path,df: pd.DataFrame):
    df.to_excel(path,index=False)
def readfolders(file):
    with open(file) as f:
        return f.read().strip().split()
rootpath='/scratch/groups/ping_group/zbai29/organize'
os.chdir(rootpath)  
foldersfile='folders'
df_file_path_out='database_out.xlsx'
assert os.path.isfile(foldersfile)
folders=readfolders(foldersfile)
DMDlist=[DMD(folder) for folder in folders]

def log_init(folder_number,prefix=''):
    logging.basicConfig(
        level=logging.INFO,
        filename=rootpath+'/%d/%sfolder_%d.log'%(folder_number,prefix,folder_number),
        format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
        datefmt='%m/%d/%Y %I:%M:%S %p',
        filemode='w',force=True)

def check_and_create_folder(number):
    subfolder=rootpath+"/%d"%number
    #if os.path.isdir(subfolder):
        #shutil.rmtree(subfolder,ignore_errors=True) 
    if not os.path.isdir(subfolder):   
        os.mkdir(subfolder)
    return subfolder
def logged(level=logging.INFO, name=None, message=None,stop=False):
    """
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__name__
        log = logging.getLogger(logname)
        @wraps(func)
        def wrapper(self,*args, **kwargs):
            log.log(level, 'startted')
            self.currentlogger=log
            res=None
            try:
                res=func(self,*args, **kwargs)
            except Exception as e:
                log.exception(e)
                self.fail=True
                if stop:
                    log.log(logging.ERROR, 'critical error, stop the program')
                    raise e
                else:
                    log.log(logging.ERROR, 'error but continue')
            log.log(level, 'finished')
            return res
        return wrapper
    return decorate

class folder_analysis():    
    def __init__(self,DMD_i,excel_i):
        self.excel_i=excel_i
        self.fail=False
        self.subfolder=check_and_create_folder(self.excel_i)
        os.chdir(self.subfolder)
        #log_init(self.excel_i)
        logging.info('Start')
        self.currentlogger=None
        self.df=pd.DataFrame()
        self.DMD_i:DMD=DMD_i
        self.df.loc[self.excel_i,'folder']=self.DMD_i.DMD_folder

    def loginit(self,prefix=''):
        log_init(self.excel_i,prefix)
    
    @logged()
    def parse_param_in(self):
        self.df.loc[self.excel_i,'Light']='%.1feV;%s;A0:%.1e'%(self.DMD_i.param.pumpE,self.DMD_i.param.pumpPoltype,self.DMD_i.param.pumpA0)
        self.df.loc[self.excel_i,'Recomb_tau']=self.DMD_i.param.tau_phenom_recomb if self.DMD_i.param.alg_phenom_recomb else '-'
    
    @logged()
    def get_Scat_State(self):
        Scat_on=True if self.DMD_i.param.alg_scatt_enable or self.DMD_i.param.alg_scatt_enable==None else False
        
        RTA_used=self.DMD_i.param.alg_phenom_relax and Scat_on
        Fully_use_RTA=RTA_used and (self.DMD_i.param.bEnd_tau-self.DMD_i.param.bStart_tau)==(self.DMD_i.lindblad_init.bTop_dm-self.DMD_i.lindblad_init.bBot_dm)

        Use_explicit_Scat= (not Fully_use_RTA) and Scat_on 

        self.df.loc[self.excel_i,'Scatt_enabled']='Off' if not Scat_on else 'On'
                
        self.df.loc[self.excel_i,'RTA']='tau %.1eps'%self.DMD_i.param.tau_phenom if RTA_used else '-'
        self.df.loc[self.excel_i,'bStart_tau']='%d'%self.DMD_i.param.bStart_tau if RTA_used else '-'
        self.df.loc[self.excel_i,'bEnd_tau']='%d'%self.DMD_i.param.bEnd_tau if RTA_used else '-'
        
        ePhDelta=read_text_from_file(self.DMD_i.lindblad_init.lindblad_folder+'/lindbladInit.out',['ePhDelta'],[2],False,float)[0]*Hatree_to_eV
        self.df.loc[self.excel_i,'Eph']='ePhDelta %.3f'%ePhDelta if Use_explicit_Scat else '-'
        self.df.loc[self.excel_i,'EE']='epsilon %.1f'%self.DMD_i.param.epsilon_background if self.DMD_i.param.eeMode and Use_explicit_Scat else '-'
        
    @logged()
    def read_lindblad_out(self):
        NkMult=read_text_from_file(self.DMD_i.lindblad_init.lindblad_folder+'/lindbladInit.out',marklist=['NkMult =']*3,locationlist=[3,4,5],stop_at_first_find=True,dtypelist=int)
        self.df.loc[self.excel_i,'K-Sampling']='NkMult %s; (%dKpoints)'%(NkMult,self.DMD_i.lindblad_init.k_number)
        
    @logged()
    def get_T_and_mu(self):
        self.DMD_i.get_mu_eV_and_T_K()
        self.df.loc[self.excel_i,'mu_eV']=self.DMD_i.mu_eV
        self.df.loc[self.excel_i,'T_K']=self.DMD_i.temperature_K
        self.currentlogger.info('mu_eV=%.3f, T_K=%.1f'%(self.DMD_i.mu_eV,self.DMD_i.temperature_K))
        
    @logged()
    def get_step_and_time(self):
        self.DMD_i.get_total_step_num_and_total_time_fs()
        self.df.loc[self.excel_i,'total_step_num']=self.DMD_i.total_step_num
        self.df.loc[self.excel_i,'total_time_fs']=self.DMD_i.total_time_fs
        
    @logged()
    def occup_time(self):
        self.DMD_i.analyze.configsetting.section_occup_time.t_max=-1
        config_result=self.DMD_i.analyze.config_result.occup_time
        occup_timestep_for_all_files=config_result.occup_timestep_for_all_files
        filelist_step=int(np.round(250/occup_timestep_for_all_files))
        self.DMD_i.analyze.configsetting.section_occup_time.filelist_step=filelist_step
        occup_t_tot=config_result.occup_t_tot
        t_max_for_occup_time=-1 if occup_t_tot<1302 else 1302
        self.DMD_i.analyze.configsetting.section_occup_time.t_max=t_max_for_occup_time
        self.DMD_i.analyze.occup_time()
        
    @logged()
    def occup_time_short_range_for_better_fit(self):
        occup_time_config_tmp=self.DMD_i.analyze.config_result.occup_time
        occup_Emax_eV=occup_time_config_tmp.occup_Emax_au/eV
        EcMin_eV=self.DMD_i.lindblad_init.energy.EcMin_eV
        self.DMD_i.analyze.configsetting.section_occup_time.plot_conduction_valence=False
        self.DMD_i.analyze.configsetting.section_occup_time.occup_time_plot_set_Erange=True
        self.DMD_i.analyze.configsetting.section_occup_time.occup_time_plot_lowE=(occup_Emax_eV+EcMin_eV)/2
        self.DMD_i.analyze.configsetting.section_occup_time.occup_time_plot_highE=occup_Emax_eV
        self.DMD_i.analyze.occup_time()
        
    @logged()
    def occup_deriv(self):
        self.DMD_i.analyze.occup_deriv()
        
    @logged()
    def current_plot(self):
        self.DMD_i.analyze.current_plot()
        
    @logged()
    def FFT_spectrum_plot(self):
        assert self.DMD_i.total_step_num!=None,'total_step_num is not set, please run get_total_step_num_and_total_time_fs() first'
        self.DMD_i.analyze.configsetting.section_FFT_spectrum_plot.Cutoff_list=max(self.DMD_i.total_step_num-1000,1)
        self.DMD_i.analyze.FFT_spectrum_plot()    
        
    @logged()
    def init_analyze(self):
        #self.DMD_i.start_analyze()
        self.DMD_i.analyze.configfile_path=rootpath+'/DMDana.ini'        

    #Read FFT-spectrum-plot-summary.csv to extact the DC current of 3 directions\
    @logged()
    def read_FFT_spectrum_plot_summary(self):
        strlist=["Cutoff",
        "FFT_integral_start_time_fs",
        "FFT_integral_end_time_fs",
        "Window_type",
        "FFT(jx_tot)(0)",
        "jx_tot_mean",
        "time(fs)",
        "FFT(jy_tot)(0)",
        "jy_tot_mean",
        "FFT(jz_tot)(0)",
        "jz_tot_mean",]
        typelist=[float,float,float,str,float,float,float,float,float,float,float]
        vallist=read_text_from_file('FFT-spectrum-plot-summary.csv',strlist,[1]*len(strlist),stop_at_first_find=False,dtypelist=typelist,sep=',')
        valdict=dict(zip(strlist,vallist))
        self.df.loc[self.excel_i,'DC_z']=valdict["FFT(jz_tot)(0)"]
        self.df.loc[self.excel_i,'DC_y']=valdict["FFT(jy_tot)(0)"]
        self.df.loc[self.excel_i,'DC_x']=valdict["FFT(jx_tot)(0)"]
        self.df.loc[self.excel_i,'Cutoff']=valdict["Cutoff"]
    @logged()
    def get_Boltzfitted(self):
        find=False
        with open('./analyze_folder_%d.log'%(self.excel_i)) as file:
            for line in file:
                if 'Boltzmann Distribution t(fs)' in line:
                    mu=line.split()[13]
                    T=line.split()[15]
                    find=True
        if find:                
            mu_Boltz,T_Boltz= float(mu),float(T)
        else:
            mu_Boltz,T_Boltz= None,None
        self.df.loc[self.excel_i,'mu_Boltz(eV)']=mu_Boltz
        self.df.loc[self.excel_i,'T_Boltz(K)']=T_Boltz
    @logged()
    def tell_system(self):
        if 'GaAs' in self.DMD_i.DMD_folder:
            self.df.loc[self.excel_i,'System']='GaAs'
        elif 'RhSi' in self.DMD_i.DMD_folder:
            self.df.loc[self.excel_i,'System']='RhSi'
        elif 'GeS' in self.DMD_i.DMD_folder or 'ges' in self.DMD_i.DMD_folder:
            self.df.loc[self.excel_i,'System']='GeS'
    @logged()
    def get_conduction_change(self):
        self.init_analyze()
        self.DMD_i.analyze.configsetting.section_occup_time.t_max=-1
        config_result=self.DMD_i.analyze.config_result.occup_time
        occup_timestep_for_all_files=config_result.occup_timestep_for_all_files
        filelist_step=int(np.round(250/occup_timestep_for_all_files))
        self.DMD_i.analyze.configsetting.section_occup_time.filelist_step=filelist_step
        occup_t_tot=config_result.occup_t_tot
        t_max_for_occup_time=-1 if occup_t_tot<1302 else 1302
        self.DMD_i.analyze.configsetting.section_occup_time.t_max=t_max_for_occup_time
        occup_t_tot_update=self.DMD_i.analyze.config_result.occup_time.occup_t_tot
        for i in self.DMD_i.analyze.config_result.occup_time.occup_selected_files:
            if DMDparser.occupation_file_class(i).time_fs< occup_t_tot_update+self.DMD_i.analyze.config_result.occup_time.occup_timestep_for_selected_file_ps*1000/2:
                lastone=i
        occupationfile=DMDparser.occupation_file_class(lastone)
        time_fs_to_eval_occupa=occupationfile.time_fs
        EcMin_eV=max(self.DMD_i.mu_eV, self.DMD_i.lindblad_init.energy.EcMin_eV)
        EvMax_eV=min(self.DMD_i.mu_eV, self.DMD_i.lindblad_init.energy.EvMax_eV)
        all_energy_list=occupationfile.data_eV[:,0]
        
        conduction_band_occupation=occupationfile.data_eV[all_energy_list>=EcMin_eV,1]
        conduction_band_energy_list=occupationfile.data_eV[all_energy_list>=EcMin_eV,0]
        valence_band_occupation=occupationfile.data_eV[all_energy_list<=EvMax_eV,1]
        valence_band_energy_list=occupationfile.data_eV[all_energy_list<=EvMax_eV,0]
        Conduction_band_Max_Change=np.max(np.abs(conduction_band_occupation-fermi(self.DMD_i.temperature_K*Kelvin,self.DMD_i.mu_eV*eV,conduction_band_energy_list)))
        Valence_band_Max_Change=np.max(np.abs(valence_band_occupation-fermi(self.DMD_i.temperature_K*Kelvin,self.DMD_i.mu_eV*eV,valence_band_energy_list)))
        self.df.loc[self.excel_i,'Max_Occupation_change_Conduction']=Conduction_band_Max_Change
        self.df.loc[self.excel_i,'Max_Occupation_change_Valence']=Valence_band_Max_Change
        self.df.loc[self.excel_i,'time_fs_to_evaluate_occupation_change']=time_fs_to_eval_occupa
        
        
    def Determine_Success_and_Save(self):
        os.chdir(rootpath)
        if not self.fail:
            self.df.loc[self.excel_i,'organize_status']='Successed'
            logging.info("Successfully finished folder %d"%self.excel_i)
        else:
            self.df.loc[self.excel_i,'organize_status']='Failed but ran to the end'
            logging.info("Parts of the script report error, but it runs until the end. (Folder %d)"%self.excel_i)
        save_database('./%d/database_out_%d.xlsx'%(self.excel_i,self.excel_i),self.df)
        return
    def analyze(self):
        self.loginit('analyze_')
        self.init_analyze()
        self.current_plot()
        self.FFT_spectrum_plot()
        self.occup_time()
        self.occup_deriv()
        self.occup_time_short_range_for_better_fit()
        #return self.df
    def summary(self):
        self.loginit('summary_')
        self.read_lindblad_out()
        self.get_Scat_State()
        self.tell_system()
        self.parse_param_in()
        self.get_step_and_time()
        self.get_T_and_mu()
        self.get_Boltzfitted()
        self.read_FFT_spectrum_plot_summary()
        self.get_conduction_change()
    def finalize_and_return(self):
        self.Determine_Success_and_Save()
        return self.df
def paralleldriver(corenum=None):
    with Pool(corenum) as p:
        res=p.starmap(subfunc,zip(DMDlist,range(excel_index_shift,len(DMDlist)+excel_index_shift)))
    os.chdir(rootpath)
    df=pd.concat(res).sort_index()
    save_database(df_file_path_out,df)
def subfunc(DMD_i,excel_i):
    tmp=folder_analysis(DMD_i,excel_i)
    tmp.summary()
    #tmp.analyze()
    return tmp.finalize_and_return()
    

In [27]:
paralleldriver(corenum=20)

/tmp/ipykernel_1909938/2793948314.py:287: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df=pd.concat(res).sort_index()


In [28]:
tmp=pd.read_excel(df_file_path_out)
for i in tmp.columns:
    print(i)

folder
K-Sampling
Scatt_enabled
RTA
bStart_tau
bEnd_tau
Eph
EE
System
Light
Recomb_tau
total_step_num
total_time_fs
mu_eV
T_K
mu_Boltz(eV)
T_Boltz(K)
DC_z
DC_y
DC_x
Cutoff
organize_status
Max_Occupation_change_Conduction
Max_Occupation_change_Valence
time_fs_to_evaluate_occupation_change


In [29]:
'''show=[]
show.append(tmp.Max_Occupation_change_Conduction)
show.append(tmp.Max_Occupation_change_Valence)
show.append(pd.Series([i.lindblad_init.energy.EcMin_eV for i in DMDlist],name='EcMin-eV'))
[i.get_mu_eV_and_T_K() for i in DMDlist]
show.append(pd.Series([i.mu_eV for i in DMDlist],name='mu'))
show.append(tmp.time_fs_to_evaluate_occupation_change)
show.append(pd.Series([i+2 for i in range(len(DMDlist))],name='id'))
pd.concat(show,axis=1)'''
part=tmp[['System','Light','mu_eV','K-Sampling','Recomb_tau','RTA','bStart_tau','bEnd_tau','Eph','EE','DC_x','DC_y',"DC_z","Max_Occupation_change_Conduction",'Max_Occupation_change_Valence','time_fs_to_evaluate_occupation_change','folder']]
part.to_excel('ana.xlsx',index=False)

In [30]:
DMDlist[0].lindblad_init.bBot_dm

0